In [1]:
import numpy as np
import matplotlib.pyplot as plt
try:
    import piplite
    await piplite.install('ipywidgets')
except: pass
from ipywidgets import interact, IntSlider, FloatSlider
import matplotlib.patches as patches
from fractions import Fraction
from math import lcm
plt.rcParams["axes.spines.bottom"] = False
plt.rcParams["axes.spines.left"] = False
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False

# The Quantum Fourier Transform

_Lorenz Kies, Mat-Nr 10021871_

The quantum Fourier transform, or QFT is a very elegant quantum equivalent of the discrete Fourier transform, which finds its use in Shor's factoring algorithm and other interesting quantum computations.

## Table of Contents:
- [1. Why a "Quantum" Fourier Transform?](#sec-1)
- [2. From Classical to Quantum](#sec-2)
  - [2.1 The Discrete Fourier Transform](#sec-2.1)
  - [2.2 The Quantum Fourier Transform](#sec-2.2)
- [3. The Quantum Fourier Transform in Shor's Algorithm](#sec-3)
- [4. Implementing the Quantum Fourier Transform](#sec-4)

<a name="sec-1"></a>

## 1. Why a "Quantum" Fourier Transform?

The most prominent application for the quantum Fourier transform (QFT) is probably as part of Shor's algorithm. One step of the algorithm involves finding the period of an integer function. Classically, this would require a lot of function evaluations. However, with a quantum computer, all of these evaluations can be performed simultaneously with one superposition containing all input values. But this does not entirely solve the problem, since only one of the input-output pairs will end up being measured, which is not really better than the classical approach. As usual, the missing piece for quantum advantage is interference. In its most general form, this means superimposing the results so a single measurement yields more information by either suppressing unwanted results or, by measuring a common relation between the results. The QFT does the latter by interfering the superpositions in such a way that measuring the frequency corresponding to the period of the period becomes more likely. Since this frequency is the interesting part, only a  few measurements are needed to know the frequency with reasonable certainty.

<a name="sec-2"></a>

## 2. From Classical to Quantum

To talk meaningfully about the QFT, a good understanding of the discrete Fourier transform (DFT) is required. A lot can be learned by examining the differences and similarities between the two.

<a name="sec-2.1"></a>

### 2.1 The Discrete Fourier Transform

Given a series $x_n$ with $n \in [0, N-1]$, the amplitude of the frequency with index $k$ can be computed as follows:

$$X_k = \frac{1}{\sqrt{N}}\sum_n^{N-1} x_k\cdot e^{-2\pi i\frac{nk}{N}} \tag{1}$$

Note, this is only **a** definition of the DFT, other definitions use a different normalization or a different sign in the exponent.

Usually only the $k \in [0, N-1]$ are of interest, this is not because the others are not reasonably defined, but because the amplitudes $X_k$ are periodic with a period of $N$. Since the DFT is its own complex conjugated inverse, it seems sensible to say that the values of the series $x_k$ should also be periodic with a period of $N$ for the coefficients to have a reasonable interpretation. This is an important observation because "discontinuities" between $x_0$ and $x_{N-1}$ will limit the accuracy of the values for low frequencies.

The expression for one output coefficient can also be written as a dot product between the series and an output-coefficient-dependent set of phase factors. The expression for all transformed coefficients can then be written as a matrix vector operation:

$$
\begin{bmatrix}X_0\\ X_1\\ X_2\\ \vdots \\ X_{N-1}\end{bmatrix} = 
\frac{1}{\sqrt{N}}
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1\\
1 & e^{-2\pi i\frac{1}{N}} & e^{-2\pi i\frac{2}{N}} & \cdots & e^{-2\pi i\frac{N-1}{N}}\\
1 & e^{-2\pi i\frac{2}{N}} & e^{-2\pi i\frac{4}{N}} & \cdots & e^{-2\pi i\frac{2(N-1)}{N}}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
1 & e^{-2\pi i\frac{N-1}{N}} & e^{-2\pi i\frac{2(N-1)}{N}} & \cdots & e^{-2\pi i\frac{(N-1)(N-1)}{N}}
\end{bmatrix}
\begin{bmatrix}x_0\\ x_1\\ x_2\\ \vdots \\ x_{N-1}\end{bmatrix}
\tag{2}
$$

With the given choice of normalization, this matrix is a unitary basis transformation. The complex conjugate of the rows (and, due to symmetry, also the columns) are the basis vectors of the new basis.

A closer look at the formula reveals that basis vectors are complex exponentials with frequencies of $\omega_k = 2\pi\cdot f_k =  \frac{2\pi k}{N}$. If the input is real, it makes more sense to interpret the second half of the frequencies as aliasing to negative frequencies because the phase difference $\phi$ between consecutive samples is greater than $\pi$ which means they can also be written as $\phi - 2\pi$. This also makes sense, considering that the continuous Fourier transform of a real signal also has conjugate symmetric negative frequencies. The frequencies are then given as:

<a name="eq-fk"></a>
$$
f_k = \begin{cases}
\frac{k}{N} & k < N/2 \\
\frac{k}{N}-1 & \text{otherwise}
\end{cases}
\tag{3}
$$

If the original series is a discretized version of a signal with a frequency _between_ two $f_k$ the transformed signal will appear as a mix between multiple frequencies. The frequency can not be accurately represented because it can not actually exist in the implicitly periodic input signal, which can only contain the base frequency $1/N$ and its harmonics. This is another justification why a "discontinuity" between the first and last points will lead to artifacts, and it will become important when using the DFT to find the period of a function using samples of the function's values.

The effect of this can be seen in the following interactive plot:

In [2]:
def dft_of_cos(T, N):
    ns = np.arange(N)
    n_ext = np.concatenate([ns[N//2:]-N, ns, ns[:N//2]+N])
    x = np.cos(2*np.pi*ns/T)
    x_ext = np.concatenate([x[N//2:], x, x[:N//2]])
    X = np.abs(np.fft.fft(x, norm="ortho"))
    X_ext = np.concatenate([X[N//2:], X, X[:N//2]])
    plt.close()
    fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 4), tight_layout=True)
    lines = axl.stem(n_ext, x_ext, label='periodic extension')
    for line in lines: line.set_color('silver')
    axl.stem(ns, x, linefmt='C0-', markerfmt='C0o', basefmt='C0-', label='signal')
    axl.legend(loc='lower left')
    axl.set_title('x(n)')
    lines = axr.stem(n_ext, X_ext, label='periodic extension')
    for line in lines: line.set_color('silver')
    axr.stem(ns, X, linefmt='C1-', markerfmt='C1o', basefmt='C1-')
    axr.set_title('|X(k)|')
    axl.set_yticks([]); axr.set_yticks([])
    
interact(dft_of_cos, T=IntSlider(min=1, max=32, step=1, value=10), N=IntSlider(min=8, max=32, step=1, value=16));

interactive(children=(IntSlider(value=10, description='T', max=32, min=1), IntSlider(value=16, description='N'…

It is clear to see that the DFT is somewhat "smeared" when the number of samples is not a multiple of the period. However, these artifacts diminish with more samples.

[[Ra10]](#ref-Ra10)

<a name="sec-2.2"></a>

### 2.2 The Quantum Fourier Transform

So what is different about the QFT? Something that might seem somewhat unintuitive at first is that the QFT is defined as an operation on a quantum state, but from the comparison to the DFT, there should be some ordered series of amplitudes that the QFT operates on. Where is this series in the quantum case? To get a better idea, it is helpful to look at a concrete example:

Consider a quantum register of three qubits, this means that there are a total of 8 basis states:

$$|0\rangle|0\rangle|0\rangle,\:|0\rangle|0\rangle|1\rangle,\:|0\rangle|1\rangle|0\rangle,\:|0\rangle|1\rangle|1\rangle,\:...,\:|1\rangle|1\rangle|1\rangle$$

or more compactly:

$$|0\rangle,\:|1\rangle,\:|2\rangle,\:|3\rangle,\:...,\:|7\rangle$$

Writing it like this already implies that the different bits have different meanings: $|0\rangle|1\rangle|0\rangle$ and $|0\rangle|1\rangle|0\rangle$ both have one bit set, but the first is interpreted as the state $|2\rangle$ and the second one as $|4\rangle$, they are interpreted differently. This interpretation as a binary number is crucial to the QFT, as the series that the transformation is performed on, are the amplitudes of the different basis states indexed by this numeral index. Since the number of basis states from $K$ qubits is $N=2^K$ the length of the series is always a power of two, which means that the QFT implicitly exploits similar symmetries to the FFT.

With all that, the QFT can be properly defined. Given a quantum state $|x\rangle = \sum_{i=0}^{N-1} x_i |i\rangle$, the coefficients of the output state $|X\rangle = \sum_{k=0}^{N-1} X_k |k\rangle$ can be determined as follows:

$$X_k = \frac{1}{\sqrt{N}}\sum_n^{N-1} x_k\cdot e^{-2\pi i\frac{nk}{N}} \tag{4}$$

which is the exact same formula as that of the DFT.

The matrix vector operation from the DFT can also be expressed in a more "quantum" way as an operator wave function operation

$$|X\rangle = F_N |x\rangle \tag{5}$$

where $F_N$ is the unitariy $N$-dimensional DFT matrix, which is made up of the $|X\rangle$ basis states as its rows.

[[Ek98]](#ref-Ek98)

<a name="sec-3"></a>

## 3. The Quantum Fourier Transform in Shor's Algorithm

Shor's algorithm for factoring an odd composite number $C$ into two factors can be roughly described in the following steps:

1. Pick a random integer $a$ with $1<a<C$ 
2. Calculate the greatest common divisor $k$ of $a$ and $C$
3. If $k\neq 1$ the number can be decomposed into $k$ and $C/k$
4. Otherwise, find the order $r$ such that $a^r \equiv 1 \mod C$ is satisfied
5. If $r$ is odd or $\gcd(a^{r/2}-1,C)=1$ go back to step 1
6. Otherwise, $C$ can be factored into $\gcd(a^{r/2}-1,C)$ and $\gcd(a^{r/2}+1,C)$

_Note: Unlike in a lot of literature, $C$ is used for the composite number to factor, this is to avoid confusion with the number of basis states $N$ from earlier._

The greatest common divisor of two numbers can be efficiently computed using Euclid's algorithm, but finding the order $r$ in step 4 classically is not faster than other factoring algorithms. This is where quantum computing and the QFT come into play.

It can easily be shown that finding the order $r>0$ is equivalent to finding the period of the following function:

$$f(x) = a^x\bmod C \tag{6}$$

This is because $f(0) = 1$ and $f(x+1) = f(x)\cdot a \mod C$ meaning that one function value directly implies the next and the first value is one. This in turn means that the function must be periodic with a period less than $C$ and the first reoccurrence of the output $1$ must appear after exactly one period.

To actually find the period, the function is evaluated on the range $[0, N-1]$ with $N=2^K$ where $K$ is the number of qubits used for the input value. This is done by evaluating the function on the superposition $|x\rangle = \frac{1}{\sqrt{N}}\sum_{i=0}^{N-1}|i\rangle$ which will result in the following state:

$$|x,f(x)\rangle =  \frac{1}{\sqrt{N}}\sum_{i=0}^{N-1}|i\rangle|f(i)\rangle \tag{7}$$

The function evaluation is performed with the help of an additional initially empty register, so that the output of the unitary function evaluation operator is two registers, one with the input values and one with the output values.

After that, the register containing the function values $f(x)$ is measured, this leaves only the terms that share the measured value $f(x_0)$. Assuming $f(x)$ does not have repeat values in one period, which is the case for modular exponentiation, the resulting superposition will have the following form:

$$|x'\rangle|f(x_0)\rangle \propto \left(|x_0\rangle + |x_0+r\rangle + |x_0+2r\rangle + ...\right)|f(x_0)\rangle \tag{8}$$

where $r$ is the period of the function. The expression is only proportional to the left side, since it needs to be renormalized. This is a periodic superposition with a period of $r$. If interpreted as a signal, this would be a series of delta impulses. By comparison to the continuous case of a Dirac comb and with the help of the definition of $f_k$ from [Eq. 3](#eq-fk) we know that the spectrum should be another series of deltas at multiples of $N/r$. However, since $N/r$ is generally not an integer, the deltas would have to be at non integer position, which is obviously not possible. The actual result is a series of somewhat "smeared" deltas, which are approximately at multiples of $N/r$. Like hinted at in [Section 2.1](#21-the-discrete-fourier-transform) the approximation should get better with more periods in the superposition. The specific value of $x_0$ is not important, it will only change the phase of the transformed series, which does not affect the probability of measuring the respective frequencies.

Finally, the transformed series is measured, and since it is approximately composed of a series of deltas, it is very likely the measurement $x_m$ will be one of the following:

$$x_m = \frac{jN}{r},\quad j \in [0, r-1] \tag{9}$$

Dividing this measured value by $N$ gives us an approximation for $j/r=x_m/N$. To recover $r$ from this, $j/r$ is approximated as $b/c$ with $b,c < C$. Since $j$ and $r$ are not necessarily coprime, $c$ will not necessarily contain all factors of $r$. To get all factors of $r$ and thereby reproduce it, the procedure is done multiple times, yielding multiple values for $c$, the order $r$ will then be the lowest common multiple of all $c$ s meaning the smallest number that can be divided by all values of $c$.

From the interactive plot, it is apparent that the procedure does not always work. There are two ways in which it fails: either one of the random measurements is not close enough to $jN/r$ which will get increasingly unlikely with more qubits, or the original fraction $x_m/N$ is not precise enough that its limited approximation can be close enough. It can be shown that the latter problem does not occur when enough qubits are used such that $K=\log_2(N)\leq2\lceil\log_2(C)\rceil+1$ is satisfied [[Ni10]](#ref-Ni10).

[[Sh99]](#ref-Sh99)

In [3]:
# try changing these parameters
a = 4 # base
C = 5*11 # composite number to be factored
K = 5 # number of qubits

def f(x):
    return a**x % C

def plot_spectrum(K: int=5):
    xs = np.arange(2**K, dtype=object) # use pythons arbitrary precision integers
    fs = f(xs)
    xps = (fs == 1) # measure "random" f(x), in this case always 1
    Xps = np.abs(np.fft.fft(xps))**2 # apply QFT
    measurements = np.random.choice(np.arange(2**K), 3, p=Xps/(Xps).sum(), replace=False) # replace is technically not correct
    fracs = [Fraction(measurement/2**K).limit_denominator(C) for measurement in measurements]

    plt.close()
    fig, (axf, axm, axX, axt) = plt.subplots(4, 1, figsize=(12, 6), tight_layout=True, sharex=True, gridspec_kw={"height_ratios":[1, 1, 1, 0.3]})
    fig.suptitle(rf"Period finding for $a={a}$, $C={C}$, $K={K}$")
    axf.stem(xs, fs, linefmt='C0-', markerfmt='C0o', basefmt='C0-')
    axf.set_title('f(x)')
    axm.stem(xs, xps, linefmt='C1-', markerfmt='C1o', basefmt='C1-')
    axm.set_title(r"$|x'\rangle|f(x_0)\rangle$")
    axX.stem(Xps, linefmt='C2-', markerfmt='C2o', basefmt='C2-')
    axX.set_title(rf"$|QFT(x')|^2$ with $N/r$ = {2**K/np.nonzero(xps)[0][1]}")
    axX.set_xlabel(r"$x$")
    axf.set_yticks([]), axm.set_yticks([]), axX.set_yticks([]), axt.set_yticks([])
    axX.tick_params(axis='x', reset=True, top=False), axt.tick_params(axis='x', bottom=False, labelbottom=False)

    axX.stem(measurements, Xps[measurements], linefmt='C3-', markerfmt='C3o', basefmt='#0000', label='measurements')
    axX.legend(loc='upper right')
    axt.annotate("    ".join([rf"$x_{{m{i}}}={m}\quad\frac{{x_{{m1}}}}{{2^{{K}}}} \approx \frac{{{f.numerator}}}{{{f.denominator}}}$" for i, (m, f) in enumerate(zip(measurements, fracs), start=1)]+[fr"$r=\mathrm{{lcm}}({','.join(str(f.denominator) for f in fracs)})={lcm(*[frac.denominator for frac in fracs])}$"]), xy=(0.5, 0.5), xycoords='axes fraction', fontsize=16, ha='center', va='center')

# plot_spectrum(8)
interact(plot_spectrum, K=IntSlider(min=4, max=10, step=1, value=5));

interactive(children=(IntSlider(value=5, description='K', max=10, min=4), Output()), _dom_classes=('widget-int…

<a name="sec-4"></a>

## 4. Implementing the Quantum Fourier Transform

Now that both the mathematics and the motivation are settled, the question of how to implement this useful tool remains.

An obvious challenge is that the transformation is done on the superpositions of joint input states, while the actual operations are done on the qubits themselves. One idea that could come to mind is to search for symmetries in the Fourier matrix, trying to find patterns of how one qubit being set or not effects the outcome of one of the output bits, or, in another sense, how each of the input qubits contribute to the output qubits. The symmetry involved in this is actually the same symmetry that is used to make the FFT so much faster than the naive implementation of the DFT.

The simplest pattern is to see how the least significant bit (LSB) of the input (bit $0$) interacts with the most significant bit (MSB) of the output (bit $K-1$). This should be the default setting in the interactive demonstration. The symbol $\omega = e^\frac{2\pi i}{N}$ which is also the first $N$ th root of unity. The powers of omega have already been reduced to their simplest form so that the exponent is never larger than $N=2^K$. Since the input vector would be multiplied from the right, resulting in some output vector, the column index is the input superposition index, while the rows correspond to the output superpositions. The blue markings signify where a certain input bit is set, and the orange markings signify where a certain output bit is set.

In [4]:
def plot_fourier_matrix(inbit=0, outbit=2, K=3):
    plt.close()
    fig, ax = plt.subplots(figsize=(6, 6), tight_layout=True)
    ax.invert_yaxis()
    N = 2**K
    fontsize = min(16, 8*16/N)
    for i in range(N):
        for j in range(N):
            ax.annotate(rf"$\omega^{{{(i*j)%N}}}$", xy=((i+0.5)/N, (j+0.5)/N), ha='center', va='center', fontsize=fontsize)
            
            # color = 'tab:blue' if i & 1<<inbit else 'tab:green'
            # ax.add_patch(patches.Rectangle(((i)/N, 1-(j+1)/N), 1/N, 1/N, fill=True, color=color, ec=None, alpha=0.5))
            color = 'tab:blue' if i & 1<<inbit else 'tab:green'
            if i & 1 << inbit:
                ax.add_patch(patches.Rectangle((i/N, j/N), 1/N, 1/N, fill=True, color='tab:blue', ec=None, alpha=0.5))
            if j & 1 << outbit:
                ax.add_patch(patches.Rectangle((i/N, j/N), 1/N, 1/N, fill=True, color='tab:orange', ec=None, alpha=0.5))
    for i in range(N):
        ax.annotate(rf"|{i:0{K}b}$\rangle$", xy=(-0.08, (i+0.5)/N), ha='center', va='center', fontsize=fontsize)
        ax.annotate(rf"$|{i:0{K}b}\rangle$", xy=((i+0.5)/N, -0.08), ha='center', va='center', fontsize=fontsize, rotation=45)
    ax.set_ylim(1, -0.12)
    ax.set_xlim(-0.12, 1)
    ax.set_aspect('equal')
    ax.set_xticks([]); ax.set_yticks([]);

interact(plot_fourier_matrix, inbit=(0, 3, 1), outbit=(0, 3, 1), K=(1, 4, 1));

interactive(children=(IntSlider(value=0, description='inbit', max=3), IntSlider(value=2, description='outbit',…

It can be seen that for states where the LSB of the input is not set (i.e. even numbers), the contribution to superpositions does not depend on the MSB of the output. If, however, the LSB is set, then the contribution to the corresponding state where the MSB is set has the opposite phase. This relation is equivalent to the Hadamard gate $\mathbf{H}$. The phase of the output is already fully explained by this, so it is independent of the other input bits.

$$\mathbf{H} = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1\end{bmatrix} \tag{10}$$

The variations in the second-most significant bit are slightly more complicated. Attempting the same approach, we see that the input's second least significant bit can not fully explain the phase of the output. For some columns, the difference is a half rotation, for others, it is a three-quarter rotation. To explain this additional quarter rotation, which only happens sometimes, we need the LSB, which can do exactly that. In the context of quantum gates, this means that the second most significant output bit is computed from a Hadamard gate of the second least significant input bit corrected by a quarter-phase gate that is controlled by the LSB denoted $\mathbf{R_2}$.

A controlled $1/2^K$ rotation $\mathbf{R_K}$ is described by the following matrix:

$$\mathbf{R_K} = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{2\pi\cdot2^{-K}}
\end{bmatrix} \tag{11}$$

It will only rotate the $|11\rangle$ state by $e^{2\pi\cdot2^{-K}}$ and leave the $|00\rangle$, $|01\rangle$ and $|10\rangle$ states as they are.

This pattern continues for all the other bits. The nth most significant output bit is computed from a Hadamard gate of the nth least significant input bit, which is then corrected by a quarter phase controlled by the n-1 th least significant bit $\mathbf{R_2}$, and an eights phase controlled by the n-2 th least significant bit $\mathbf{R_3}$ and so on.

The resulting gate setup can be seen in the following figure, it also includes swap gates so that the output is **not** in bit-reversed order. However, this is not really necessary to implement in hardware, as the measurement can simply be reinterpreted in bit-reversed order.

<center>

</center>


Using this strategy, the QFT can be constructed with $K(K-1)/2$ quantum gates, which is on the order of $O(K^2)$. This is considerably less complex than the FFT, which needs on the order of $O(K2^K)$ gates or operations to compute the DFT on a series of $N=2^K$ values.

[[Ek98]](#ref-Ek98)

Practically, the number of gates needed is even lower, this is because the impact of the controlled phase gates with very small phases is negligible, and an approximation of the QFT is enough for Shor's algorithm to work [[Sh99]](#ref-Sh99).

## References

<a name="ref-Ra10"></a>
[Ra10] Rao, K. R., Kim, D. N., & Hwang, J. J. (2010). Fast Fourier transform: algorithms and applications (Vol. 32). Dordrecht: Springer.

<a name="ref-Ek98"></a>
[Ek98] Ekert, A., & Jozsa, R. (1998). Quantum algorithms: entanglement–enhanced information processing. Philosophical Transactions of the Royal Society of London. Series A: Mathematical, Physical and Engineering Sciences, 356(1743), 1769-1782. [arXiv](https://arxiv.org/abs/quant-ph/9803072)

<a name="ref-Sh99"></a>
[Sh99] Shor, P. W. (1999). Polynomial-time algorithms for prime factorization and discrete logarithms on a quantum computer. SIAM review, 41(2), 303-332. [arXiv](https://arxiv.org/pdf/quant-ph/9508027.pdf)

<a name="ref-Ni10"></a>
[Ni10] Nielsen, M. A., & Chuang, I. L. (2010). Quantum computation and quantum information. Cambridge university press.